In [ ]:
!unzip /content/drive/MyDrive/dataset.zip

In [2]:
import numpy as np
import pandas as pd
import librosa
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import IPython.display as ipd
from tensorflow.keras.optimizers import SGD
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.array(active_events)

In [3]:
l = os.listdir('/content/X')
l.sort()
filename_data = pd.DataFrame(l , columns = ['filename'])

l = os.listdir('/content/Y')
l.sort()
event_data= pd.DataFrame(l , columns = ['eventname'])


In [4]:
def r_shape(data):
  data = np.reshape(data,(64000))
  return data

In [5]:
data=np.zeros((10000,64000))
labels=np.zeros((10000,11))
for i,row in filename_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/X'),str(row["filename"]))
  d=np.load(file_name)
  data[i]=r_shape(d)

In [6]:
for i,row in event_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/Y'),str(row["eventname"]))
  e = np.load(file_name)
  e=eventroll_to_multihot_vector(e)
  labels[i]=e

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size=0.20,random_state=42)

In [8]:
x_train=data
y_train=labels
del(data)
del(labels)

In [ ]:
model = Sequential()
model.add(Dense(500, activation='relu', input_shape=(64000,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='sigmoid'))


In [ ]:
history = model.fit(x_train, y_train, epochs=15, validation_data= (x_test, y_test),batch_size=32)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
for i in range(2000):
  for z in range(11):
    if y_pred[i][z]>=0.45:
      y_pred[i][z]=1
    else:
      y_pred[i][z]=0


In [ ]:
from sklearn.metrics import f1_score
k=f1_score(y_test, y_pred, average=None)
print(k)
k=np.mean(k,axis=0)
print(k)

In [ ]:
del(x_train)
del(y_train)